# Generating data for Case studies

we will generate all the necessary data for the paper in this workbook
This means:
- 3 case studies, each with their respective test and reference sessions
- 2D matching: 
  - show examples of good and bad matches with the inlier lines and the resulting projection errors
  - show the results of each method separately and weighted average over all the test images, this results
    - each test returns one estimated pose per test image and or mesh (each time there is a search for the best match)
  - compare the estimated global pose against the actual one
- 3D matching:
  - show example of good and bad matches
  - show the correspondences in 3D space with the inlier percent and error

In [ ]:
#Importing the relevant modules
import numpy as np


import session
from imagematch import ImageMatch

## Case Selection:
Pick one of the 3 cases underneath to apply all the functions to the relevant case

### Case 0: Kamer
Bedroom for fast testing

In [ ]:
globalPos = np.array([0,0,0])
accuraccy = 10

testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/TestSessions/0_Kamer"
resultPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/Results/0_Kamer"
# Getting the test session from the device/path
testSession = session.Session().from_path(testSessionPath)

### Case 1: Campus
The technologiecampus Gent, with the focus on the 3 different rooms that were tested: Dubo, Beton, grond
The main test here is locating withing a large dataset

In [ ]:
globalPos = np.array([0,0,0])
accuraccy = 10

testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/TestSessions/1_Campus"
resultPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/Results/1_Campus"
# Getting the test session from the device/path
testSession = session.Session().from_path(testSessionPath)

### Case 2: House
The house of Maarten Bassier which has had a renovation

In [ ]:
globalPos = np.array([0,0,0])
accuraccy = 10

testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/TestSessions/2_House"
resultPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/Results/2_House"
# Getting the test session from the device/path
testSession = session.Session().from_path(testSessionPath)

### Case 3: LivingLab
The living lab house which has been recorded over all the phases of the building

In [ ]:
globalPos = np.array([0,0,0])
accuraccy = 10

testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/TestSessions/1_Campus"
resultPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/Results/1_Campus"
# Getting the test session from the device/path
testSession = session.Session().from_path(testSessionPath)

## Reference Data Selection

### Multiple reference sessions:
With the given global position, only the relevant sessions are stored and parsed.

In [ ]:
#Locating the Reference folder with all the reference sesisons
refSessionsPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/RefSessions"
refSessions = session.find_close_sessions(refSessionsPath, globalPos, accuraccy)

### Single Session selection
This also has an option to select one single session to compare against


In [ ]:

# Getting the test session from the device/path
testSession = session.Session().from_path(testSessionPath)
#test with 1 reference session
refSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42"
refSessions = [session.Session().from_path(refSessionPath)]

## 2D Alignment

### Showing all the matches

this prints out all the match results with all the reference images

In [ ]:
from imagematch import ImageMatch

results = []

#test every test image against every reference image and get the match object to check the error and confidence
for refSession in refSessions:
    print("Matching reference session:", refSession.sessionId)

    for testImage in testSession.imageTransforms:
        print("Test Image:", testImage.id, "has a matchError of:")
        for refImage in refSession.imageTransforms:
            newMatch = ImageMatch(refImage, testImage)
            newMatch.find_matches() # find the best matches 
            results.append(newMatch)
            print(newMatch.matchError, "With image:", refImage.id)

### Good and bad match examples
This prints out a very good and very bad match based on the match score, also shows the correspondences

In [ ]:
# hand pick the good and bad matches
testImage = testSession.imageTransforms[1]
goodImage = testSession.imageTransforms[2]
badImage = testSession.imageTransforms[6]

bestMatch = ImageMatch(testImage, goodImage)
worstMatch = ImageMatch(testImage, badImage)


In [ ]:
#find the best and worst match in the list of all the images

imageNr = len(testSession.imageTransforms)

bestMatch: ImageMatch = None
worstMatch: ImageMatch = None

for i in range(0,imageNr):
    for j in range(i+1, imageNr):
        print(i, " vs. ", j)
        newMatch = ImageMatch(testSession.imageTransforms[i], testSession.imageTransforms[j])
        newMatch.find_matches() # find the best matches
        if(bestMatch == None or bestMatch.matchError > newMatch.matchError):
            bestMatch = newMatch
        if(worstMatch == None or worstMatch.matchError < newMatch.matchError):
            worstMatch = newMatch

    

In [ ]:
bestMatch.get_essential_matrix()
goodmatches = bestMatch.draw_image_matches()
goodinliers = bestMatch.draw_image_inliers()

worstMatch.get_essential_matrix()
badmatches = worstMatch.draw_image_matches()
badinliers = worstMatch.draw_image_inliers()

In [ ]:
from imagematch import ImageMatch
from matplotlib import pyplot as plt
import cv2

import os



def show_and_save_matches(title, image, error):
    plt.rcParams['figure.dpi'] = 300
    plt.axis('off')
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title + "\n Error:" + str(error))
    plt.savefig(os.path.join(resultPath,"match examples", title))
    plt.show()

show_and_save_matches('Bad Image Match', badmatches, worstMatch.matchError)
show_and_save_matches('Bad Image Inliers', badinliers, worstMatch.matchError)
show_and_save_matches('Good Image Match', goodmatches, bestMatch.matchError)
show_and_save_matches('Good Image Inliers', goodinliers, bestMatch.matchError)


### Pose estimations method 1 (cross reference matching)
Using 2 separate matches to calculate the final pose

In [ ]:
import positioning2d as pos2d

for refsession in refSessions:

    for testImage in testSession.imageTransforms:
        print("Test Image:", testImage.id)
        R,t, confidence = pos2d.cross_reference_matching(testImage, refSession)

        testOriginRot = testImage.get_rotation_matrix().T @ R
        testOrgininPos = - testOriginRot @ t
        testSession.add_pose_guess(refSession, testOriginRot, testOrgininPos, confidence)
        print("Estimated pose wit confidence:", confidence)
        print("R: \n", testOriginRot, "\n t:\n", testOrgininPos.T)



### Pose estimation method 2 (incremental)
using 2 connected matches to find an estimation

In [ ]:
# we will test every test image against all the reference images and their best match

import positioning2d as pos2d

for refsession in refSessions:

    for testImage in testSession.imageTransforms:
        print("Test Image:", testImage.id, "matched with:")
        R,t, confidence = pos2d.incremental_matching(testImage, refSession)

        testOriginRot = testImage.get_rotation_matrix().T @ R
        testOrgininPos = - testOriginRot @ t
        testSession.add_pose_guess(refSession, testOriginRot, testOrgininPos, confidence)
        print("Estimated pose wit confidence:", confidence)
        print("R: \n", testOriginRot, "\n t:\n", testOrgininPos.T)

### Pose estimation method 3 (raycasting)
using the 3D scene to get the session scale

In [ ]:
# we will test every test image against all the reference images and their best match

import positioning2d as pos2d

for refsession in refSessions:

    for testImage in testSession.imageTransforms:
        print("Test Image:", testImage.id, "matched with:")
        R,t, confidence = pos2d.raycast_matching(testImage, refSession) #get the rotation and translation with the pnp point algorithm

        testOriginRot = testImage.get_rotation_matrix().T @ R
        testOrgininPos = - testOriginRot @ t
        testSession.add_pose_guess(refSession, testOriginRot, testOrgininPos, confidence)
        print("Estimated pose wit confidence:", confidence)
        print("R: \n", testOriginRot, "\n t:\n", testOrgininPos.T)

## 3D alignment

This section covers the 3D alignment process

### Good and bad matches examples

shows good and bad matches based on the error and inlier percent

In [ ]:
#displaying corespondances
import open3d as o3d
import positioning3d as pos3d
from geometrymatch import GeometryMatch
from geometrytransform import GeometryTransform
import copy

import numpy as np

pcd1Path = "/Users/jellevermandere/Downloads/Users/travis/build/nmellado/Super4PCS/Super4PCS-v1.1.3-osx-clang++/assets/roomPart.obj"
pcd2Path = "/Users/jellevermandere/Downloads/Users/travis/build/nmellado/Super4PCS/Super4PCS-v1.1.3-osx-clang++/assets/roomHolo.obj"
pcd3Path = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42/mesh-2022-01-14 09-23-42.obj"

testPcd = GeometryTransform().from_path(pcd1Path)
goodPcd = GeometryTransform().from_path(pcd2Path)
badPcd = GeometryTransform().from_path(pcd3Path)

goodmatch = GeometryMatch(testPcd, goodPcd, 0.05)
goodresult = goodmatch.get_transformation()
goodlineset = goodmatch.draw_matches()
print("goodRMSE:",goodresult.inlier_rmse)
print("goodFitness:",goodresult.fitness)
badmatch = GeometryMatch(testPcd, badPcd, 0.05)
badresult = badmatch.get_transformation()
badlineset = badmatch.draw_matches()
print("badRMSE:",badresult.inlier_rmse)
print("badFitness:",badresult.fitness)

#pos3d.show_geometries([testPcd.get_voxel_pcd(),  goodPcd.get_voxel_pcd(),badPcd.get_voxel_pcd(), goodlineset, badlineset], True)





In [ ]:
goodmovedPcd = copy.deepcopy(goodPcd.geometry)
goodmovedPcd.transform(goodresult.transformation)
goodmovedPcd.compute_vertex_normals()

pos3d.show_geometries([testPcd.get_voxel_pcd(), goodmovedPcd,goodPcd.get_voxel_pcd(), goodlineset], True)

In [ ]:
badmovedPcd = copy.deepcopy(badPcd.geometry)
badmovedPcd.transform(badresult.transformation)
badmovedPcd.compute_vertex_normals()

pos3d.show_geometries([testPcd.get_voxel_pcd(), badmovedPcd,badPcd.get_voxel_pcd(), badlineset], True)

### Method 1: FPFH feature matching

the first method involves matching 

In [ ]:
import session
import positioning3d as pos3d

# Getting the test session from the device/path
#testSessionPath = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-09-47"
testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42"
testSession = session.Session().from_path(testSessionPath)
#test with 1 reference session
refSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42"
refSession = session.Session().from_path(refSessionPath)

pos3d.compare_session(testSession, refSession, 0.05)


## All the Estimations

In [ ]:
import positioning3d as pos3d
import open3d as o3d
# show all the estimations in 3d space from black to white, by confidence

points = []
colors = []

for estimation in testSession.estimations:
    points.append(estimation[1])
    colors.append(estimation[2] * [1,1,1])

estimatedPoints = o3d.geometry.PointCloud()
estimatedPoints.points = o3d.utility.Vector3dVector(points)
estimatedPoints.colors = o3d.utility.Vector3dVector(colors)

pos3d.show_geometries([estimatedPoints])

